In [0]:
!pip install sentence_transformers
from google.colab import drive
from zipfile import ZipFile
import torch
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
drive.mount('/content/gdrive')

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 573kB 6.6MB/s 
     |████████████████████████████████| 3.7MB 13.8MB/s 
     |████████████████████████████████| 890kB 43.6MB/s 
     |████████████████████████████████| 1.0MB 9.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=07d9176a848143ef2380b53743fa703870907ba700a05e4824c9abe9a884bcda
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=e2f4bf6892028c1970ff8b544aba22baccd9c1896effd04b56ab58857bad9b9e
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sentence-transformers sacremoses
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee

In [ ]:
# read config
config
with open("../config.json") as f:
  config = json.load(f)["locations"]

In [0]:
zippedFile = config["zipped_patents"]
with ZipFile(zippedFile, 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
def get_embeddings_from_file(file_name):
  with open(f"patent_text/{file_name}", "r") as f:
    lines = [line.strip("\n ") for line in f.readlines()]
    try:
      sentence_embeddings = (
          np.mean(model.encode(lines), axis=0).reshape((768, 1))
      )
    except:
      sentence_embeddings = np.zeros((768,1))
    return sentence_embeddings

100%|██████████| 245M/245M [00:11<00:00, 21.3MB/s]


In [0]:
folder = "patent_text/"
files_list = sorted(os.listdir(folder))
files = {idx: f"{file_name}" for idx, file_name in enumerate(files_list)}
start_idx, end_idx = 0, len(files)
embeddings = np.empty(end_idx-start_idx, dtype=object)
# embeddings = np.array([])
print(embeddings.shape)
i = 0
for idx in tqdm(range(start_idx, end_idx)):
  file = files[idx]
  if idx % 5000 == 0 and idx > start_idx:
    print(idx)
    # print("result: ", result)
    # print("embeddings length", len(embeddings))
    df = pd.DataFrame({
          "patents": [files[x] for x in range(start_idx, idx+1)], 
          "embedding": embeddings[:i+1]
        })
    df.to_json(config["bert_embeddings"])
  # p.apply_async(get_embeddings_from_file, [file])
  result = get_embeddings_from_file(file) 
  embeddings[i] = result
  i += 1

df = pd.DataFrame({
      "patents": [files[x] for x in range(start_idx, end_idx)], 
      "embedding": embeddings
    })

df.to_json(config["bert_embeddings"])